## Take Home Analytics Excercises
### Caitlin Ruble

"This is an exercise to gauge your familiarity with coding and think through analysis implications. You'll find a prompt and two columns below: 'Coding' and 'Questions'. For the 'Coding' portion please submit the SQL query you would run to figure out the answer. The tables are spelled out in the subsequent tabs and are populated with both a definition of the table/columns and SAMPLE data. The 'Questions' portion asks you to think through the implications of a given result.

Feel free to submit in whatever format you're most comfortable with."


### **Prompt** : You work for J.Crew. J.Crew has just acquired Madewell and you're working with a team that is trying to understand the Madewell business and opportunities to cross-sell to loyalty members.

### My approach: 
I'm going to use pandas and pandasql to load the sample databases and ensure my queries are working. I will write the MySQL versions of the queries as commented-out code cells within this notebook, and I will also save these to an .sql file to be saved in the same repository. These should be considered my final answers, though the SQLite queries will show my experimentation process and the results of running these queries with the provided sample data. Please note that sqlite has some subtle syntax differences from MySQL, which I have done my best to account for.

First I'll need to load the appropriate libraries and set the provided example database tables up as DataFrames. Because they're so small and on Excel sheets that also contain text, I'll simply manually create the DataFrames.

## Data Loading

In [1]:
import pandas as pd
import pandasql as ps
import datetime as dt

In [2]:
#load sample tables as Pandas DataFrames, and add a few more example rows in checkouts and checkout_items for comprehensive testing:

checkouts = pd.DataFrame({'customer_id' : [1, 2, 3, 4, 1, 2, 3, 4], 'cart_id' : [56, 57, 58, 59, 60, 61, 62, 63], 'date' : ['2021-01-02', '2021-01-02', '2021-01-08', '2021-02-01', '2021-02-15', '2021-02-05', '2021-02-28', '2021-03-08']})
checkouts['date'] = pd.to_datetime(checkouts['date'], format=  '%Y-%m-%d') #format the dates as datetime objects so they can be used for queries/calculations
print('checkouts:')
print(checkouts)

checkout_items = pd.DataFrame({'cart_id' : [56, 56, 56, 56, 57, 58, 58, 59, 60, 61, 62, 63], 'item' : ['White Shirt', 'Mens Short', 'Cotton jersey short', 'Slim-fit jean', 'White Shirt', 'Mens Short', 'Cotton jersey short', 'Slim-fit jean', 'White Shirt', 'White Shirt', 'Mens Short', 'Cotton jersey short'], 'quantity': [4, 1, 1, 2, 3, 2, 5, 1, 2, 2, 5, 3], 'price_per_unit_cents': [1200, 600, 500, 900, 1200, 600, 500, 900, 1200, 1200, 600, 500]})
print('checkout_items:')
print(checkout_items)

customers = pd.DataFrame({'customer_id':[1, 2, 3, 4], 'name': ['John','Cindy','George','Alice'], 'family_size': [1, 6, 2, 3]})
print('customers:')
print(customers)

checkouts:
   customer_id  cart_id       date
0            1       56 2021-01-02
1            2       57 2021-01-02
2            3       58 2021-01-08
3            4       59 2021-02-01
4            1       60 2021-02-15
5            2       61 2021-02-05
6            3       62 2021-02-28
7            4       63 2021-03-08
checkout_items:
    cart_id                 item  quantity  price_per_unit_cents
0        56          White Shirt         4                  1200
1        56           Mens Short         1                   600
2        56  Cotton jersey short         1                   500
3        56        Slim-fit jean         2                   900
4        57          White Shirt         3                  1200
5        58           Mens Short         2                   600
6        58  Cotton jersey short         5                   500
7        59        Slim-fit jean         1                   900
8        60          White Shirt         2                  1200
9       

## Excercise 1: 

**SQL**: What is monthly sales by customer type? Customer type defined as Single (1 family size), Couple (2 family size), Family (3-5), Large Family (6+)

**Analysis Q** Why would J.Crew care about this? If Madewell's sales is high and increasing for Single family types, how should Madewell think about that? What else would you want to know?

**My Answer**: This is a basic function that would allow J.Crew/Madewell to track which types of customers are making purchases, when those purchases and being made, how many purchases each group is making, the quantity of items bought and the amount of money spent by each group. This paints a picture of sales behavior which can be leveraged for all sorts of things: marketing campaigns, sales predictions, supply chain forecasting, etc. 

If Madewell's sales are high and increasing for Single family types, it shows us that the current marketing, onboarding, product quality & selection and price point are all *working* for this family type right now. Madewell could lean into this by increasing their marketing toward Single people, aiming to cast a wider net and onboard more new customers in this group. It would be important for Madewell to consider which products are most popular with this group and ensure their supply chain is equipped to handle the greater demand that would be the goal of such a marketing campaign. This could be investigated by tweaking the SQL query to include the specifics of which type of items are being purchased by this group over time: their quantity, cost, net profit to the company, etc. We could analyze the sales trend data to uncover insights into product popularity and quantify the bottom line for J.Crew/Madewell. In this way, we could uncover the types and quantity of products the company needs to make to keep up with demand, use data to inform future product development and identify marketing and J.Crew/Madewell crossover opportunities in this customer base.

On the other hand, Madewell could consider the disparities between the Single family type and the other family types; perhaps there is market potential to capture more customers who fall into one of the other family types. I would probably start by comparing the monthly sales behavior of each group and identify the group(s) with the *most* similarity in purchasing behavior to the Single family type. Then, I'd investigate what the disparities are: do we simply have fewer customers of this group? Are they buying the same products or different products? Would it be reasonable for Madewell to increase production and marketing of the products this group *is* buying? 

In [3]:
#This is the full query, as it would be written in MYSQL or PostgreSQL

# WITH
#  checkouts_fam AS (
#    SELECT 
#    ch.customer_id, 
#    ch.cart_id, 
#    DATE_FORMAT(ch.date, '%m-%Y') AS month_year, 
#      CASE 
#        WHEN cu.family_size = 1 THEN 'Single'
#        WHEN cu.family_size = 2 THEN 'Couple'
#        WHEN cu.family_size > 5  THEN 'Large Family'
#        ELSE 'Family'
#      END AS customer_type,
#     cu.family_size
#    FROM checkouts AS ch 
#    LEFT JOIN customers AS cu 
#      ON ch.customer_id = cu.customer_id),
#  transactions AS (
#    SELECT 
#    cart_id, 
#    SUM((price_per_unit_cents * quantity)/100) AS trans_value_USD,
#    SUM(quantity) AS n_items_purchased
#    FROM checkout_items 
#    GROUP BY cart_id)
# SELECT 
# customer_type, 
# month_year, 
# COUNT(cf.cart_id) AS total_monthly_cart_checkouts, 
# SUM(tr.n_items_purchased) AS total_monthly_items_puchased,
# SUM(tr.trans_value_USD) AS total_monthly_trans_value_USD
# FROM checkouts_fam AS cf
# LEFT JOIN transactions AS tr 
#  ON cf.cart_id = tr.cart_id 
# GROUP BY customer_type, month_year
# ORDER BY cf.family_size, month_year;

In [4]:
#check the contents of cte1
print('checkouts_fam')
print(ps.sqldf("SELECT ch.customer_id, ch.cart_id, strftime('%m-%Y', ch.date) AS month_year, CASE WHEN cu.family_size = 1 THEN 'Single' WHEN cu.family_size = 2 then 'Couple' WHEN cu.family_size >2 AND cu.family_size <6 then 'Family' ELSE 'Large Family' END AS customer_type, cu.family_size FROM checkouts AS ch LEFT JOIN customers AS cu ON ch.customer_id = cu.customer_id"))

checkouts_fam
   customer_id  cart_id month_year customer_type  family_size
0            1       56    01-2021        Single            1
1            2       57    01-2021  Large Family            6
2            3       58    01-2021        Couple            2
3            4       59    02-2021        Family            3
4            1       60    02-2021        Single            1
5            2       61    02-2021  Large Family            6
6            3       62    02-2021        Couple            2
7            4       63    03-2021        Family            3


In [5]:
#Test the contents of cte2
print('transactions')
print(ps.sqldf("SELECT cart_id, SUM((price_per_unit_cents * quantity)/100) AS trans_value_USD, SUM(quantity) AS n_items_purchased FROM checkout_items GROUP BY cart_id"))


transactions
   cart_id  trans_value_USD  n_items_purchased
0       56               77                  8
1       57               36                  3
2       58               37                  7
3       59                9                  1
4       60               24                  2
5       61               24                  2
6       62               30                  5
7       63               15                  3


In [6]:

print('Question1 Full Query')
full_query = "WITH checkout_fam AS (SELECT ch.customer_id, ch.cart_id, strftime('%m-%Y', ch.date) AS month_year, CASE WHEN cu.family_size = 1 THEN 'Single' WHEN cu.family_size = 2 then 'Couple' WHEN cu.family_size >2 AND cu.family_size <6 then 'Family' ELSE 'Large Family' END AS customer_type, cu.family_size FROM checkouts AS ch LEFT JOIN customers AS cu ON ch.customer_id = cu.customer_id), transactions AS (SELECT cart_id, SUM((price_per_unit_cents * quantity)/100) AS trans_value_USD, SUM(quantity) AS n_items_purchased FROM checkout_items GROUP BY cart_id) SELECT cf.customer_type, cf.month_year, COUNT(cf.cart_id) AS total_monthly_transactions, SUM(tr.n_items_purchased) AS total_monthly_items_puchased, SUM(tr.trans_value_USD) AS total_monthly_trans_value_USD FROM checkout_fam AS cf JOIN transactions AS tr ON cf.cart_id = tr.cart_id GROUP BY cf.customer_type, cf.month_year ORDER BY cf.family_size, cf.month_year"

print(ps.sqldf(full_query))


Question1 Full Query
  customer_type month_year  total_monthly_transactions  \
0        Single    01-2021                           1   
1        Single    02-2021                           1   
2        Couple    01-2021                           1   
3        Couple    02-2021                           1   
4        Family    02-2021                           1   
5        Family    03-2021                           1   
6  Large Family    01-2021                           1   
7  Large Family    02-2021                           1   

   total_monthly_items_puchased  total_monthly_trans_value_USD  
0                             8                             77  
1                             2                             24  
2                             7                             37  
3                             5                             30  
4                             1                              9  
5                             3                             15  
6

## Excercise 2: 

**SQL**: What is the average cart value by acquisition year cohort (e.g., what is the average cart value for customers acquired in 2019, 2020, 2021, etc.)

**Analysis Q**: 
You find that average cart value is going down for later acquisition cohorts v. earlier — What are some hypotheses for why this is happening and what analysis would you run to test those hypotheses?

**My Answer**:
 Some hypotheses: 
 1. Later aquisition cohorts may have less disposable income; e.g. they could be recent college grads vs. more-established working professionals from earlier cohorts. An analysis of the average cart value *over time* for each cohort would reveal any trends in support of this hypothesis. For instance, if we found that the average cart value of earlier cohorts increased over time, it would support that each cohort tends to buy a greater value of product over time. A line plot with average cart value over time, grouped by year cohort and normalized for the amount of time since signing up, would be a really nice way to visualize this. Analyzing demographic data for each cohort would also be helpful for this analysis, if we can get it. I'd ideally want to look at age, education, profession, income bracket, location (and cost of living in location), etc to test out this hypothesis and gain insights on customer behavior.
 
 2. Later cohorts may be biased toward spending *less* at the same time, but how often are they buying? Does this group tend to purchase a lower cart value at a time, but make those purchases more often? How does this behavior (and the net profit to the company) compare to earlier cohorts? I would run an analysis to compare average number of cart purchases per month for each cohort to analyse purchase frequency behavior. I could multiply the avg frequency of purchases by the avg cart value to evaluate how each cohort is contributing to company profit. These figures could be compared between the cohort groups over time to evaluate any real differences in behavior. If we found, for instance, that later groups **do** prefer to purchase less at a time but more frequently, that would give the business valuable insight into how we may be able to increase the purchase volume for this type of customer--would offering free shipping on a lower cart value encourage this type of customer to buy smaller quantities even more frequently, e.g.? We could run an A|B test to find out.

 3. If the frequency of purchases is heavily influenced by the cohort year (i.e. earlier cohorts are buying less frequently as time goes on vs. new cohorts), a change to the overall cost of the products over time could explain the observation. An analysis of average product cost along with average cart value over time could indicate whether the observation that later aquisition cohorts are spending less in each purchase is a reflection of changes to the cost-structure over time. I would couple this with an analysis of purchase frequency over time for each cohort, to evaluate the trends in cohort purchashing behavior as a function of how long they've been a customer, similar to in point 1, above.

 4. Does our data cut off mid-year, thereby telling an incomplete story about the most recent cohort? If cart values go up significantly around the Holidays, but we don't have that data for the most recent cohort year, the simple explanation could be that we're not comparing apples to apples and need to exclude the current year's cohort from the analysis.

In [7]:
# #full SQL query

# WITH
#    cart_vals AS(
#        SELECT 
#        ch.customer_id,
#        ch.cart_id, 
#        SUM((ci.price_per_unit_cents * quantity)/100) AS cart_value_USD 
#        FROM checkouts as ch
#        LEFT JOIN checkout_items AS ci
#        ON ci.cart_id = ch.cart_id
#        GROUP BY ch.cart_id
#        ),
#    cohorts AS(
#        SELECT 
#        customer_id,
#        MIN(YEAR(date)) AS year_cohort
#        FROM checkouts
#        GROUP BY customer_id
#        ) 
# SELECT 
# co.year_cohort,
# AVG(cv.cart_value_USD) AS avg_cart_value_USD
# FROM cart_vals AS cv
# LEFT JOIN cohorts AS co
# ON cv.customer_id = co.customer_id
# GROUP BY co.year_cohort
# ORDER BY co.year_cohort;


In [8]:
print("cart_vals")
print(ps.sqldf("SELECT ch.customer_id, ch.cart_id, SUM((ci.price_per_unit_cents * quantity)/100) AS cart_value_USD FROM checkouts AS ch LEFT JOIN checkout_items AS ci ON ci.cart_id = ch.cart_id GROUP BY ch.cart_id"))

cart_vals
   customer_id  cart_id  cart_value_USD
0            1       56              77
1            2       57              36
2            3       58              37
3            4       59               9
4            1       60              24
5            2       61              24
6            3       62              30
7            4       63              15


In [9]:
#note: using "month cohort" instead of "year cohort" for the purposes of testing in sqlite. Note also that the datetime methods look a little different

print("cohorts")
print(ps.sqldf("SELECT customer_id, MIN(strftime('%m-%Y', date)) AS month_cohort FROM checkouts GROUP BY customer_id"))

cohorts
   customer_id month_cohort
0            1      01-2021
1            2      01-2021
2            3      01-2021
3            4      02-2021


In [10]:
print(ps.sqldf("WITH cart_vals AS (SELECT ch.customer_id, ch.cart_id, SUM((ci.price_per_unit_cents * quantity)/100) AS cart_value_USD FROM checkouts AS ch LEFT JOIN checkout_items AS ci ON ci.cart_id = ch.cart_id GROUP BY ch.cart_id), cohorts AS (SELECT customer_id, MIN(strftime('%m', date)) AS month_cohort FROM checkouts GROUP BY customer_id) SELECT co.month_cohort, AVG(cv.cart_value_USD) AS average_cart_value_USD FROM cart_vals AS cv LEFT JOIN cohorts AS co ON cv.customer_id = co.customer_id GROUP BY co.month_cohort ORDER BY co.month_cohort"))



  month_cohort  average_cart_value_USD
0           01                    38.0
1           02                    12.0


## Excercise 3:

**SQL**: For each customer, what is the difference between the value of their first order and the second order?

**Analysis Q**: Is this a useful metric? Why or why not? Feel free to suggest alternative metrics that could be interesting here!

**My Answer**:
I'd say this *might* be a useful metric, depending on the question we're trying to answer. If we want to know whether users spend more, less or the same amount on their second order compared to their first order, this is the right metric for that! This could help us understand user behavior to drive business decisions, and could be a concrete metric for A|B Testing, where we might assess whether a new marketing campaign, product launch, UX interface, brick-and-mortar store feature etc. encourages new users to spend more money on their second purchase, and thereby might be more likely to be happy repeat customers.

We might also look at the amount of time between a customer's first and second purchases and the proportion of new customers who make a second purchase as an alternative way to assess our success with making new customers repeat customers. 

One of the problems with this metric is that it is most useful for new customers; we can use the data from the older customers as a historic reference. It might be interesting to look at these same metrics but with respect to the customers' *last two* purchases as a means of analyzing shopping behavior for existing customers. In this case our lens would probably be focused on strategies to increase profits through customer retention.

#full Sql query:

<!-- 
WITH 
cart_vals AS(
        SELECT 
        ch.customer_id,
        ch.date AS time_of_purchase,
        ch.cart_id, 
        SUM((ci.price_per_unit_cents * quantity)/100) AS cart_value_USD,
        COUNT(ch.cart_id) OVER (PARTITION BY customer_id ORDER BY ch.date) AS order_number 
        FROM checkouts as ch
        LEFT JOIN checkout_items AS ci
        ON ci.cart_id = ch.cart_id
        GROUP BY ch.cart_id
        ),
differences AS(
        SELECT customer_id, 
        order_number,
        cart_value_USD - LEAD(cart_value_USD) OVER(PARTITION BY customer_id ORDER BY order_number) AS difference 
        FROM cart_vals 
        WHERE order_number <= 2)
SELECT 
customer_id,
FIRST_VALUE(difference) OVER (PARTITION BY customer_id ORDER BY order_number) as value_of_first_order_minus_value_of_second_order_USD,
FROM differences
GROUP BY customer_id -->

In [11]:
#I'm just printing these here again for ease of viewing while constructing my query
print('checkouts:')
print(checkouts)

print('checkout_items:')
print(checkout_items)

print('customers:')
print(customers)

checkouts:
   customer_id  cart_id       date
0            1       56 2021-01-02
1            2       57 2021-01-02
2            3       58 2021-01-08
3            4       59 2021-02-01
4            1       60 2021-02-15
5            2       61 2021-02-05
6            3       62 2021-02-28
7            4       63 2021-03-08
checkout_items:
    cart_id                 item  quantity  price_per_unit_cents
0        56          White Shirt         4                  1200
1        56           Mens Short         1                   600
2        56  Cotton jersey short         1                   500
3        56        Slim-fit jean         2                   900
4        57          White Shirt         3                  1200
5        58           Mens Short         2                   600
6        58  Cotton jersey short         5                   500
7        59        Slim-fit jean         1                   900
8        60          White Shirt         2                  1200
9       

In [12]:
print("cart_vals")
print(ps.sqldf("SELECT ch.customer_id, ch.date AS time_of_purchase, ch.cart_id, SUM((ci.price_per_unit_cents * quantity)/100) AS cart_value_USD, COUNT(ch.cart_id) OVER (PARTITION BY customer_id ORDER BY ch.date) AS order_number FROM checkouts as ch LEFT JOIN checkout_items AS ci ON ci.cart_id = ch.cart_id GROUP BY ch.cart_id ORDER BY customer_id, date"))
cart_vals= ps.sqldf("SELECT ch.customer_id, ch.date AS time_of_purchase, ch.cart_id, SUM((ci.price_per_unit_cents * quantity)/100) AS cart_value_USD, COUNT(ch.cart_id) OVER (PARTITION BY customer_id ORDER BY ch.date) AS order_number FROM checkouts as ch LEFT JOIN checkout_items AS ci ON ci.cart_id = ch.cart_id GROUP BY ch.cart_id ORDER BY customer_id, date")

cart_vals
   customer_id            time_of_purchase  cart_id  cart_value_USD  \
0            1  2021-01-02 00:00:00.000000       56              77   
1            1  2021-02-15 00:00:00.000000       60              24   
2            2  2021-01-02 00:00:00.000000       57              36   
3            2  2021-02-05 00:00:00.000000       61              24   
4            3  2021-01-08 00:00:00.000000       58              37   
5            3  2021-02-28 00:00:00.000000       62              30   
6            4  2021-02-01 00:00:00.000000       59               9   
7            4  2021-03-08 00:00:00.000000       63              15   

   order_number  
0             1  
1             2  
2             1  
3             2  
4             1  
5             2  
6             1  
7             2  


In [13]:
print('differences')
print(ps.sqldf("SELECT customer_id, order_number, cart_value_USD - LEAD(cart_value_USD) OVER(PARTITION BY customer_id ORDER BY order_number) AS difference FROM cart_vals WHERE order_number <= 2"))

differences = ps.sqldf("SELECT customer_id, order_number, cart_value_USD - LEAD(cart_value_USD) OVER(PARTITION BY customer_id ORDER BY order_number) AS difference FROM cart_vals WHERE order_number <= 2")

differences
   customer_id  order_number  difference
0            1             1        53.0
1            1             2         NaN
2            2             1        12.0
3            2             2         NaN
4            3             1         7.0
5            3             2         NaN
6            4             1        -6.0
7            4             2         NaN


In [14]:
print('final result')
print(ps.sqldf("SELECT customer_id, FIRST_VALUE(difference) OVER (PARTITION BY customer_id ORDER BY order_number) as value_of_first_order_minus_value_of_second_order_USD FROM differences GROUP BY customer_id"))

final result
   customer_id  value_of_first_order_minus_value_of_second_order_USD
0            1                                               53.0   
1            2                                               12.0   
2            3                                                7.0   
3            4                                               -6.0   


In [15]:
#All together now!
print(ps.sqldf("WITH cart_vals AS (SELECT ch.customer_id, ch.date, ch.cart_id, SUM((ci.price_per_unit_cents * quantity)/100) AS cart_value_USD, COUNT(ch.cart_id) OVER (PARTITION BY customer_id ORDER BY ch.date) AS order_number FROM checkouts as ch LEFT JOIN checkout_items AS ci ON ci.cart_id = ch.cart_id GROUP BY ch.cart_id ORDER BY customer_id, date), differences AS (SELECT customer_id, order_number, cart_value_USD - LEAD(cart_value_USD) OVER(PARTITION BY customer_id ORDER BY order_number) AS difference FROM cart_vals WHERE order_number <= 2) SELECT customer_id, FIRST_VALUE(difference) OVER (PARTITION BY customer_id ORDER BY order_number) as value_of_first_order_minus_value_of_second_order_USD FROM differences GROUP BY customer_id "))

   customer_id  value_of_first_order_minus_value_of_second_order_USD
0            1                                                 53   
1            2                                                 12   
2            3                                                  7   
3            4                                                 -6   


I look forward to continuing to learn how to make my SQL queries quick and efficient!